In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import sys
sys.path.append("/content/drive/MyDrive/Colab Notebooks/instquality/")

import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/instquality/")

In [7]:
import pandas as pd
import numpy as np
import zipfile
import warnings

warnings.filterwarnings("ignore")

In [8]:
from functions.load import *

datasets = load()

(efw, # economic freedom of the world, fraser institute, 1970-2021
 fiw, # freedom in the world, freedom house, 2006-2023
 ief, # index of economic freedom, heritage foundation, 1995-2023
 p5d, # polity5d, polity project, 1960-2020
 pts, # political terror scale, 1976-2022
 wgi, # world governance indicators, world bank, 1996-2022
 wb, # institutional quality data + macro data (control), world bank, 1960-2023
 tgt # 10y sovereign yields (target), OECD and world bank, 1960-2024

) = (datasets[k] for k in ["efw", "fiw", "ief", "p5d", "pts", "wgi", "wb", "tgt"])

Loading files/EFW.xlsx
Loading files/FIW.xlsx
Loading files/IEF.csv
Loading files/P5D.xls
Loading files/PTS.xlsx
Loading files/WGI.xlsx
Loading files/WB_DATA.zip
Loading files/OECD.csv


In [9]:
from functions.merge import *

dfs = [efw, fiw, ief, p5d, pts, wgi, wb, tgt]
names = ['efw', 'fiw', 'ief', 'p5d', 'pts', 'wgi', 'wb', 'tgt']

conc = create_concordance(dfs)

for i, name in enumerate(names):
    dfs[i] = apply_concordance(dfs[i], conc, name)

efw, fiw, ief, p5d, pts, wgi, wb, tgt = dfs

cdfs = [df.drop_duplicates(subset=['country', 'year'])
        for df in dfs
        #if df is not wb
]

merged = merge(cdfs, conc)

Removed from efw: []
Removed from fiw: ['Abkhazia' 'Antigua and Barbuda' 'Chechnya' 'Crimea' 'Eastern Donbas'
 'Gaza Strip' 'Indian Kashmir' 'Israeli Occupied Territories' 'Kiribati'
 'Marshall Islands' 'Micronesia' 'Nagorno-Karabakh' 'Northern Cyprus'
 'Pakistani Kashmir' 'Palau' 'Samoa' 'Solomon Islands' 'Somaliland'
 'South Ossetia' 'St. Kitts and Nevis' 'St. Lucia'
 'St. Vincent and the Grenadines' 'Tibet' 'Tonga' 'Transnistria' 'Tuvalu'
 'Vanuatu']
Removed from ief: ['Kiribati' 'Micronesia' 'Saint Lucia' 'Saint Vincent and the Grenadines'
 'Samoa' 'Solomon Islands' 'Tonga' 'Vanuatu']
Removed from p5d: ['Solomon Islands' 'South Vietnam' 'Vietnam North' 'Yemen North'
 'Yemen South']
Removed from pts: ['Antigua and Barbuda' 'Crimea' 'European Union' 'Gaza (Hamas)'
 'Israel in Occupied Territories' 'Israel in pre-1967 borders' 'Kiribati'
 'Marshall Islands' 'Micronesia, Federated States of' 'Palau'
 'Russia-Occupied Areas (Ukraine)' 'Saint Kitts and Nevis' 'Saint Lucia'
 'Saint Vincen

In [10]:
# save

merged.to_pickle("files/inst.dat")